<a href="https://pymt.readthedocs.io"><img style="float: right" src="images/pymt-logo-header-text.png"></a>

# Coupling models

In this tutorial we will learn how to:
* Couple two models
* Use the PyMT grid mapper

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

## Set up and initialize two models

For this example we will couple and landscape evolution model, *Child*, with a seascape evolution model, *Sedflux*. Although these are two very different models, notice that within *PyMT* they look and operate in a familiar way.

In [ ]:
from pymt import MODELS

First, set up and initialize *Child*...

In [ ]:
child = MODELS.Child()

child_in, child_dir = child.setup(
    "_child",
    grid_node_spacing=500.0,
    grid_x_size=40000.0,
    grid_y_size=20000.0,
    run_duration=1e6,
)

child.initialize(child_in, dir=child_dir)

...then set up and initialize *Sedflux*

In [ ]:
sedflux = MODELS.Sedflux3D()

sedflux_in, sedflux_dir = sedflux.setup(
    "_sedflux",
    river_bed_load_flux=0.0,
    river_suspended_load_concentration_0=0.1,
    river_suspended_load_concentration_1=0.1,
    run_duration=1e6 * 365.0,
)

sedflux.initialize(sedflux_in, dir=sedflux_dir)

Before continuing, we need to sync up values of surface elevation for each of the models. For *Child*, the variable name for this is *land_surface__elevation*.

In a similar way as we did in the previous tutorial, we'll set a shoreline in *Child*. In this case, though, we'll put the shoreline at *x=20km*.

In [ ]:
gid = child.var["land_surface__elevation"].grid

x, y = child.grid_x(gid), child.grid_y(gid)
z = child.get_value("land_surface__elevation")

In [ ]:
x_shore = 20000.0
z[np.where(x > x_shore)] += 100.0
z[np.where(x <= x_shore)] -= 100.0

child.set_value("land_surface__elevation", z)

## Map values between grids

As we've seen, the *Child* grid is unstructured. The *Sedflux* grid, however, is uniform rectilinear. We can see this with *quick_plot*. For *Sedflux*, the relevent parameter is *bedrock_surface__elevation*.

In [ ]:
sedflux.quick_plot(
    "bedrock_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r"
)

We've set the values for the initial elevation for *Child*, we need to map these to the values of *Sedflux*. We set these values in *Sedflux* using the usual **set_value** method, only this time we use the `mapfrom` keyword.

The below code, *bedrock_surface__elevation* values of *Sedflux* by mapping values of *land_surface__elevation* of child.

In [ ]:
sedflux.set_value(
    "bedrock_surface__elevation", mapfrom=("land_surface__elevation", child)
)

To double-check how the mapping turned out, plot the elevations for each model.

In [ ]:
child.quick_plot(
    "land_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r"
)

In [ ]:
sedflux.quick_plot("bedrock_surface__elevation", vmin=-200, vmax=200, cmap="BrBG_r")

Although *Child* will provide sediment to *Sedflux*, *Sedflux* still requires values for the river characteristics as it enters the ocean.

In [ ]:
sedflux.set_value("channel_exit_water_flow__speed", 1.2)
sedflux.set_value("channel_exit_x-section__mean_of_width", 400.)
sedflux.set_value("channel_exit_x-section__mean_of_depth", 4.)

## Run a coupled simulation!

We now run the time loop. For each time step, we:
* update *Child*
* set *Child*'s sediment flux into *Sedflux*
* update *Sedflux*
* set *Sedflux*'s seafloor elevations into *Child*

In [ ]:
now = child.time
times = np.arange(now, now + 1000, 1.0)

sedflux.update()
child.update()

for t in tqdm(times):
    child.update_until(t, units="years")

    sedflux.set_value("channel_water_sediment~bedload__mass_flow_rate", mapfrom=child)
    sedflux.update_until(t, units="years")

    z = child.get_value("land_surface__elevation")
    child.set_value(
        "land_surface__elevation",
        mapfrom=("land-or-seabed_sediment_surface__elevation", sedflux),
        nomap=np.where(z > 0.0),
    )

In [ ]:
child.quick_plot(
    "land_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r"
)